In [1]:
from ptyrodactyl import simul, tools, workflows
import jax
import jax.numpy as jnp

W1212 20:36:11.445143  547618 cuda_executor.cc:1802] GPU interconnect information not available: INTERNAL: NVML doesn't support extracting fabric info or NVLink is not used by the device.
W1212 20:36:11.453286  547617 cuda_executor.cc:1802] GPU interconnect information not available: INTERNAL: NVML doesn't support extracting fabric info or NVLink is not used by the device.
W1212 20:36:11.453820  547615 cuda_executor.cc:1802] GPU interconnect information not available: INTERNAL: NVML doesn't support extracting fabric info or NVLink is not used by the device.
W1212 20:36:11.454655  547616 cuda_executor.cc:1802] GPU interconnect information not available: INTERNAL: NVML doesn't support extracting fabric info or NVLink is not used by the device.
W1212 20:36:11.455199  547621 cuda_executor.cc:1802] GPU interconnect information not available: INTERNAL: NVML doesn't support extracting fabric info or NVLink is not used by the device.
W1212 20:36:11.455648  547619 cuda_executor.cc:1802] GPU int

In [2]:
import numpy as np
import matplotlib.pyplot as plt

In [3]:
jax.devices()

[CudaDevice(id=0),
 CudaDevice(id=1),
 CudaDevice(id=2),
 CudaDevice(id=3),
 CudaDevice(id=4),
 CudaDevice(id=5),
 CudaDevice(id=6),
 CudaDevice(id=7)]

In [10]:
data_file = "../../../data/frame_001.xyz"

In [11]:
mos2_data: tools.CrystalData = simul.parse_crystal(data_file)
mos2_data.positions.shape

FileNotFoundError: [Errno 2] No such file or directory: '../../../data/frame_001.xyz'

In [ ]:
x_pos = mos2_data.positions[:, 0]
y_pos = mos2_data.positions[:, 1]
z_pos = mos2_data.positions[:, 2]

In [ ]:
jnp.max(x_pos), jnp.min(x_pos), jnp.max(y_pos), jnp.min(y_pos), jnp.max(z_pos), jnp.min(z_pos)


In [ ]:
voltage_kv = 60
cbed_aperture_mrad = 5.0
cbed_extent_mrad = 50.0
cbed_shape = (256, 256)
real_space_pixel_size_ang = 0.1
slice_thickness = 1.0


In [ ]:
# Use fewer scan positions for faster testing (40x40 = 1600 positions)
# Increase to 150x150 for full resolution once testing is complete
yy, xx = jnp.meshgrid(jnp.linspace(-50, 100, 40), jnp.linspace(-50, 100, 40))
scan_positions = jnp.asarray((yy.ravel(), xx.ravel())).T
print(f"Number of scan positions: {scan_positions.shape[0]}")
scan_positions

In [ ]:
cbed_simulation = workflows.crystal2stem4d(
    crystal_data=mos2_data, 
    scan_positions=scan_positions, 
    voltage_kv=voltage_kv, 
    cbed_aperture_mrad=cbed_aperture_mrad, 
    cbed_extent_mrad=cbed_extent_mrad, 
    cbed_shape=cbed_shape, 
    real_space_pixel_size_ang=real_space_pixel_size_ang, 
    slice_thickness=slice_thickness,
    force_parallel=True
    )